In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/appbites/Desktop/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [5]:

project = hopsworks.login(engine="python")
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()

AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
# The station location is hard-coded here. When Github Actions are fixed, this will be just SENSOR_LOCATION_JSON
secret = secrets.get_secret("SENSOR_LOCATION_JSON_ecke-taborstra-e---glockengasse").value
print(f"🔐 Using secret: {secret}")

station_cfg = json.loads(secret)

# Extract parameters
STATION_UID = station_cfg["station_uid"]
COUNTRY = station_cfg["country"]
CITY = station_cfg["city"]
STREET = station_cfg["street"]
URL = station_cfg["url"]
LAT = float(station_cfg["latitude"])
LON = float(station_cfg["longitude"])

today = datetime.date.today()

print(f"🌍 {CITY}, {COUNTRY}")
print(f"📍 {STREET} [UID={STATION_UID}]")
print(f"🔗 {URL}")
print(f"📡 Coordinates: {LAT}, {LON}")




2025-11-12 19:44:59,381 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-12 19:44:59,389 INFO: Initializing external client
2025-11-12 19:44:59,389 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-12 19:45:00,911 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1289364
🔐 Using secret: {"station_uid": 2860, "country": "Austria", "city": "Vienna", "street": "Ecke Taborstra\u00dfe - Glockengasse", "url": "https://aqicn.org/station/@2860/", "latitude": "48.216739", "longitude": "16.380918"}
🌍 Vienna, Austria
📍 Ecke Taborstraße - Glockengasse [UID=2860]
🔗 https://aqicn.org/station/@2860/
📡 Coordinates: 48.216739, 16.380918


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [6]:
# Retrieve feature groups
stations_air_quality_daily = fs.get_feature_group(
    name='stations_air_quality_daily',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [8]:
import requests
import pandas as pd

feed_url = f"https://api.waqi.info/feed/@{STATION_UID}/"
aq_today_df = util.get_pm25(
    aqicn_url=feed_url,
    country=COUNTRY,
    city=CITY,
    street=STREET,
    day=today,
    AQI_API_KEY=AQICN_API_KEY
)

aq_today_df

,pm25,country,city,street,date,url
0,42.0,Austria,Vienna,Ecke Taborstraße - Glockengasse,2025-11-12,https://api.waqi.info/feed/@2860/


In [9]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [10]:
hourly_df = util.get_hourly_weather_forecast(CITY, LAT, LON)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = CITY
daily_df

Coordinates 48.25°N 16.5°E
Elevation 176.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-12,6.40,0.0,9.585739,124.286934,Vienna
1,2025-11-13,6.90,0.0,9.000000,143.130020,Vienna
2,2025-11-14,7.75,0.0,15.021105,134.029068,Vienna
3,2025-11-15,9.15,0.0,3.240000,360.000000,Vienna
4,2025-11-16,13.55,0.0,5.483356,156.801376,Vienna
5,2025-11-17,7.40,0.9,19.694992,329.215912,Vienna
6,2025-11-18,6.75,0.0,5.991594,122.735222,Vienna


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [12]:
# Insert new data
aq_today_df["station_uid"] = STATION_UID
stations_air_quality_daily.insert(aq_today_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: stations_air_quality_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1289364/jobs/named/stations_air_quality_daily_1_offline_fg_materialization/executions


(Job('stations_air_quality_daily_1_offline_fg_materialization', 'SPARK'), None)

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-12 19:50:37,042 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1289364/fs/1278019/fg/1680714


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1289364/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-12 19:51:12,620 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-12 19:51:15,816 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
